In [1]:
import numpy as np
import cv2
import os

from sklearn.model_selection import train_test_split

from Autoencoder import *

In [2]:
cap = cv2.VideoCapture('bunny_video.mp4')
print ("[+] Loading Video")

index = 0
successs,img = cap.read()

path = r'C:\Users\Saad.LAKES\Desktop\Autoencoders\Video_Frames'
path = path.replace('\\','/')
path_old = os.getcwd()
os.chdir(path)
names=[]

while successs:
    cv2.imwrite('frame_{}.jpg'.format(index),img)
    names.append('frame_{}.jpg'.format(index))
    successs,img = cap.read()
#     print('Reading the new frame:',successs)
    index+=1
    
os.chdir(path_old)

[+] Loading Video


In [3]:
## the total images are 739
def read_imgs(path,names):
    path = path.replace('\\','/')
    path_old=os.getcwd()
    os.chdir(path)
    img =[]
    for name in names:
        temp = cv2.imread(name)
        img.append(cv2.resize(temp,(720,720)))
    return img

In [4]:
images = np.array(read_imgs(path,names))

In [5]:
images = images.astype('float')/255.

In [6]:
cv2.imshow(names[1],images[1])
cv2.waitKey(0)
cv2.destroyAllWindows()

In [7]:
X_train, X_test = train_test_split(images, test_size=0.1, random_state=42)
print(f'The shape of X_train is {X_train.shape}')
print(f'The shape of X_test is {X_test.shape}')

The shape of X_train is (665, 720, 720, 3)
The shape of X_test is (74, 720, 720, 3)


In [8]:
IMG_SHAPE = images.shape[1:]
embed_size = 32
encoder, decoder = build_autoencoder(IMG_SHAPE, embed_size,name='Simple')

inp = Input(IMG_SHAPE)
code = encoder(inp)
reconstruction = decoder(code)

autoencoder = Model(inp,reconstruction)
autoencoder.compile(optimizer='adam', loss='mse',metrics=['mae'])

print(autoencoder.summary())

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 720, 720, 3)]     0         
_________________________________________________________________
sequential (Sequential)      (None, 32)                49766432  
_________________________________________________________________
sequential_1 (Sequential)    (None, 720, 720, 3)       51321600  
Total params: 101,088,032
Trainable params: 101,088,032
Non-trainable params: 0
_________________________________________________________________
None


In [11]:
history = autoencoder.fit(x=X_train, y=X_train, epochs=200,validation_split=0.2,batch_size=32)

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

MemoryError: Unable to allocate 3.08 GiB for an array with shape (532, 720, 720, 3) and data type float32